#  Jupyter Notebook for *A Deep Learning Algorithm for High-Dimensional Exploratory Item Factor Analysis*

### Journal: Psychometrika
### Authors: Christopher J. Urban and Daniel J. Bauer
### Affil.: L. L. Thurstone Psychometric Laboratory in the Dept. of Psychology and Neuroscience, UNC-Chapel Hill
### E-mail: cjurban@live.unc.edu

This notebook is for conducting amortized importance-weighted variational inference for exploratory IFA.

First, import necessary packages, then runtime parameters.

In [ ]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import sys
import timeit
import re
import os
import csv
from pathlib import Path
from functools import reduce
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from pylab import *
import subprocess

from utils import *
from helper_layers import *
from base_class import *
from mirt_vae import *
from read_data import *
from cross_validation import *
from simulations import *

# Re-import some packages to reload functions without needing to restart the kernel.
import sys, importlib
importlib.reload(sys.modules["utils"])
importlib.reload(sys.modules["helper_layers"])
importlib.reload(sys.modules["base_class"])
importlib.reload(sys.modules["mirt_vae"])
importlib.reload(sys.modules["read_data"])
importlib.reload(sys.modules["cross_validation"])
importlib.reload(sys.modules["simulations"])

from utils import *
from helper_layers import *
from base_class import *
from mirt_vae import *
from read_data import *
from cross_validation import *
from simulations import *

# Suppress scientific notation.
np.set_printoptions(suppress = True)

# Print full arrays.
np.set_printoptions(threshold = sys.maxsize)

# If CUDA is available, use it.
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
kwargs = {"num_workers" : 1, "pin_memory" : True} if cuda else {}

# Set log intervals.
ipip_log_interval = 200
sim_log_interval = 75

base_path = "" # Base path goes here.

## Empirical Data Analysis

### IPIP-FFM Analyses

Make IPIP-FFM data loaders.

In [ ]:
ipip_filename = base_path + "data/ipip_ffm/ipip_ffm_recoded.csv"

# Full data set.
ipip_loader = torch.utils.data.DataLoader(
    ipip_ffm_dataset(csv_file = ipip_filename,
                     which_split = "full",
                     transform = to_tensor()),
    batch_size = 32, shuffle = True, **kwargs)

# Test data set.
ipip_test_loader = torch.utils.data.DataLoader(
    ipip_ffm_dataset(csv_file = ipip_filename,
                     which_split = "test-only",
                     transform = to_tensor()),
    batch_size = 32, shuffle = True, **kwargs)


# Train data set.
ipip_train_loader = torch.utils.data.DataLoader(
    ipip_ffm_dataset(csv_file = ipip_filename,
                     which_split = "train-only",
                     transform = to_tensor()),
    batch_size = 32, shuffle = True, **kwargs)

#### Random Starts

Fit model with many random starts and save fitted values.

In [ ]:
# Make directory to save results.
res_path = base_path + "results/ipip_ffm/random_starts/"
Path(res_path).mkdir(parents = True, exist_ok = True)

for rep in range(100):
    print("Starting replication", rep)
    
    # Set random seeds.
    torch.manual_seed(rep)
    np.random.seed(rep)

    # Initialize model.
    start = timeit.default_timer()
    ipip_vae = MIRTVAEClass(input_dim = 250,
                            inference_model_dims = [130],
                            latent_dim = 5,
                            n_cats = [5]*50,
                            learning_rate = 5e-3,
                            device = device,
                            log_interval = ipip_log_interval,
                            steps_anneal = 1000)

    # Fit model.
    ipip_vae.run_training(ipip_loader, ipip_test_loader, iw_samples = 5)
    stop = timeit.default_timer()
    
    # Save run time.
    Path(res_path + "run_times/").mkdir(parents = True, exist_ok = True)
    np.savetxt(res_path + "/run_times/run_time_" + str(rep) + ".txt",
               np.array([stop - start]), 
               delimiter = ",",
               fmt = "%f")

    # Save model loadings.
    Path(res_path + "loadings/").mkdir(parents = True, exist_ok = True)
    np.savetxt(res_path + "loadings/loadings_" + str(rep) + ".txt",
               ipip_vae.model.loadings.weight.data.numpy(),
               fmt='%f')

    # Save model intercepts.
    Path(res_path + "intercepts/").mkdir(parents = True, exist_ok = True)
    np.savetxt(res_path + "intercepts/intercepts_" + str(rep) + ".txt",
               ipip_vae.model.intercepts.bias.data.numpy(),
               fmt = "%f")
    
    # Save model log-likelihood.
    ll = -ipip_vae.bic(ipip_test_loader,
                       iw_samples = 5000)[1]
    Path(res_path + "lls/").mkdir(parents = True, exist_ok = True)
    np.savetxt(res_path + "lls/ll_" + str(rep) + ".txt",
               np.array([ll]), 
               delimiter = ",",
               fmt = "%f")
    
    print("Stored results for replication", rep)

# Rotate factor loadings.
subprocess.call(base_path + "code/r/rotate_ipip_ffm.R")

Compute predicted approximate log-likelihood for different latent dimensions P.

In [ ]:
ll_ls = []
latent_dims = np.arange(1, 11).tolist()
for latent_dim in latent_dims:
    print("Starting fitting for P =", latent_dim)
    
    # Set random seeds.
    torch.manual_seed(1)
    np.random.seed(1)

    # Initialize model.
    start = timeit.default_timer()
    ipip_vae = MIRTVAEClass(input_dim = 250,
                            inference_model_dims = [int((250 + 2 * latent_dim) / 2)],
                            latent_dim = latent_dim,
                            n_cats = [5]*50,
                            learning_rate = 5e-3,
                            device = device,
                            log_interval = ipip_log_interval,
                            steps_anneal = 1000)

    # Fit model.
    ipip_vae.run_training(ipip_train_loader, ipip_test_loader, iw_samples = 5)
    stop = timeit.default_timer()
    
    print("Model fitted, run time =", stop - start, "seconds")
    
    # Save model log-likelihood.
    torch.manual_seed(1)
    np.random.seed(1)
    start = timeit.default_timer()
    ll_ls.append(-ipip_vae.bic(ipip_test_loader, iw_samples = 5000)[1])
    stop = timeit.default_timer()
    
    print("LL stored, computed in", stop - start, "seconds")
    
for idx, dim in enumerate(latent_dims):
    print("Latent dimension =", dim, "LL =", ll_ls[idx])

Save scree plot.

In [ ]:
fig, ax = plt.subplots(constrained_layout = True)
fig.set_size_inches(5, 5, forward = True)
fig.set_size_inches(5, 5, forward = True)
ax.axvline(x = 5, color = "gray", linestyle = "--")
ax.plot(np.arange(1, 11).tolist(), [ll for ll in ll_ls], "k-o")
ax.set_xticks(np.arange(1, 11).tolist())
ax.set_ylabel("Predicted Approximate Negative Log-Likelihood")
ax.set_xlabel("Number of Factors")
fig.suptitle("Approximate Log-Likelihood Plot for IPIP-FFM Data")
fig.show()

pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/scree_plot_ipip_ffm.pdf")
pdf.savefig(fig, dpi = 300)
pdf.close()

Compute mean RMSEs for fitted values across random starts.

In [ ]:
# Read in log-likelihood filenames.
res_path = base_path + "results/ipip_ffm/random_starts/"
ll_filenames = os.listdir(res_path + "lls/")

# Read in fitted values.
lls        = [np.loadtxt(res_path + "lls/ll_" + str(num) + ".txt", dtype = float).item() for
              num in range(len(ll_filenames))]
idxs       = [num for num in range(len(lls)) if not np.isnan(lls[num])]
loadings   = [np.loadtxt(res_path + "rotated_loadings/rotated_loadings_" + str(num) + ".txt", dtype = float) for
              num in range(len(ll_filenames)) if not np.isnan(lls[num])]
intercepts = [np.loadtxt(res_path + "intercepts/intercepts_" + str(num) + ".txt", dtype = float) for
              num in range(len(ll_filenames)) if not np.isnan(lls[num])]
covs       = [np.loadtxt(res_path + "covs/cov_" + str(num) + ".txt", dtype = float) for
              num in range(len(ll_filenames)) if not np.isnan(lls[num])]
run_times  = [np.loadtxt(res_path + "run_times/" + "run_time_" + str(num) + ".txt",
                         dtype = float, skiprows = 1).sum() for
              num in range(len(ll_filenames)) if not np.isnan(lls[num])]

# Obtain reference values.
best_idx = lls.index(min(lls))
ref_ldgs, ref_ints, ref_cov = loadings.pop(best_idx), intercepts.pop(best_idx), covs.pop(best_idx)
best_idx = idxs.pop(best_idx)

# Find first 100 converged solutions.
n_keeps    = 100
keeps      = [all([c > 0.98 for c in permuted_congruence(ref_ldgs, ldgs, mean = False)]) for ldgs in loadings]
loadings   = [ldgs for ldgs, keep in zip(loadings, keeps) if keep][0:n_keeps]
intercepts = [ints for ints, keep in zip(intercepts, keeps) if keep][0:n_keeps]
covs       = [cov for cov, keep in zip(covs, keeps) if keep][0:n_keeps]
run_times  = [run_time for run_time, keep in zip(run_times, keeps) if keep][0:n_keeps]

# Calculate loadings RMSEs.
loadings_biases  = [permuted_biases(ref_ldgs, ldgs) for ldgs in loadings]
loadings_rmses   = np.sqrt(reduce(np.add, [bias**2 for bias in loadings_biases]) / len(loadings_biases))

# Calculate intercepts RMSEs.
n_cats = [5]*50
intercepts_biases = [normalize_ints(ref_ints, ref_ldgs, n_cats) - normalize_ints(ints, ldgs, n_cats) for
                     ints, ldgs in zip(intercepts, loadings)]
intercepts_rmses  = np.sqrt(reduce(np.add, [bias**2 for bias in intercepts_biases]) / len(intercepts_biases))

# Calculate covariance RMSEs.
covariance_biases = [cov_biases(ref_cov, perm_cov, ref_ldgs, perm_ldgs) for
                     perm_cov, perm_ldgs in zip(covs, loadings)]
covariance_rmses  = np.sqrt(reduce(np.add, [bias**2 for bias in covariance_biases]) / len(covariance_biases))

print("Mean Loadings RMSE: {:.3f} ({:.3f})".format(np.mean(loadings_rmses),
                                                   np.std(loadings_rmses)))
print("Mean Intercepts RMSE: {:.3f} ({:.3f})".format(np.mean(intercepts_rmses),
                                                     np.std(intercepts_rmses)))
print("Mean Covariance Matrix RMSE: {:.3f} ({:.3f})".format(np.mean(covariance_rmses),
                                                            np.std(covariance_rmses)))
print("Mean Run Time: {:.2f} ({:.2f})".format(np.mean(run_times),
                                              np.std(run_times)))

#### Saving Model Results

Save loadings estimates in normal metric and factor correlation matrix for paper.

In [ ]:
res_path   = base_path + "results/ipip_ffm/random_starts/"
loadings   = np.loadtxt(res_path + "rotated_loadings/rotated_loadings_" + str(best_idx) + ".txt",
                        dtype = float)
intercepts = np.loadtxt(res_path + "intercepts/intercepts_" + str(best_idx) + ".txt",
                        dtype = float)
cov        = np.loadtxt(res_path + "covs/cov_" + str(best_idx) + ".txt",
                        dtype = float)

# Save loadings.
np.savetxt(base_path + "results/ipip_ffm/loadings_for_paper.txt",
           np.around(normalize_loadings(loadings), decimals = 2), 
           delimiter = " & ",
           fmt = "%1.2f")

# Save loadings heatmap.
loadings = np.loadtxt(base_path + "results/ipip_ffm/loadings_for_paper.txt",
                      delimiter = "&")[:, [0, 3, 2, 4, 1]]
c = pcolor(invert_factors(loadings))
set_cmap("gray_r")
colorbar() 
c = pcolor(invert_factors(loadings), edgecolors = "w", linewidths = 1, vmin = 0) 
xlabel("Factor")
ylabel("Item")
xticks(np.arange(loadings.shape[1]) + 0.5,
       ["EXT", "EST", "AGR", "CON", "OPN"])
yticks(np.array([10, 20, 30, 40, 50]) - 0.5, ["10", "20", "30", "40", "50"])
plt.gca().invert_xaxis()
suptitle("IPIP-FFM Factor Loadings", y = 0.93)
savefig(base_path + "figures/loadings_heatmap.pdf")

# Save correlation matrix.    
np.savetxt(base_path + "results/ipip_ffm/cors_for_paper.txt",
           np.around(cov, decimals = 2),
           delimiter = " & ",
           fmt = "%1.2f")

Save loadings matrix, intercept vectors, factor mean vector, and factor covariance matrix for simulations.

In [ ]:
# Obtain a sparse loadings matrix and save it.
ldgs = np.around(ref_ldgs, decimals = 1)
nonzero_row_idxs = [[] for _ in range(ldgs.shape[1])] 
factor_idx = 0 
for row in range(0, ldgs.shape[0]): 
    nonzero_row_idxs[factor_idx].append(np.where(abs(ldgs[row, :]) == max(abs(ldgs[row, :])))[0].item()) 
    factor_idx += (row % 10 == 9)
nonzero_row_idxs = [Counter(ls).most_common()[0][0] for ls in nonzero_row_idxs]
factor_idx = 0 
for row in range(0, ldgs.shape[0]): 
    ldgs[row, np.delete(np.arange(ldgs.shape[1]), nonzero_row_idxs[factor_idx])] = 0
    factor_idx += (row % 10 == 9)
np.savetxt(base_path + "data/simulations/loadings.txt",
           ldgs, 
           delimiter = ",",
           fmt = "%1.1f")

# Save intercepts.
np.savetxt(base_path + "data/simulations/intercepts.txt",
           np.around(ref_ints, decimals = 1),
           delimiter = ",",
           fmt = "%1.1f")

# Save covariance matrix.    
np.savetxt(base_path + "data/simulations/lv_cov.txt",
           np.around(ref_cov, decimals = 2),
           delimiter = ",",
           fmt = "%1.2f")

## Simulated Data Analyses

### Simulation 1: Evaluating Consistency and Efficiency of the Variational Estimator

Fit simulated data sets.

In [ ]:
# Load data generating parameters.
ref_loadings   = torch.from_numpy(np.loadtxt(base_path + "data/simulations/loadings.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_intercepts = torch.from_numpy(np.loadtxt(base_path + "data/simulations/intercepts.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_cov        = torch.from_numpy(np.loadtxt(base_path + "data/simulations/lv_cov.txt",
                                  dtype = float,
                                  delimiter = ",")).float()

# Set simulation cell parameters.
n_obs_ls         = [500, 1000, 2000, 10000]
iw_samples_ls    = [1, 5, 25]

# Loop across simulation cells.
for obs_idx, n_obs in enumerate(n_obs_ls):
    for iw_idx, iw_samples in enumerate(iw_samples_ls):
        # Print simulation cell.
        print("Starting replications for simulation cell " + str(int(iw_idx + 1)) + ", " + str(int(obs_idx + 1)))

        # Try to load replication results.
        sim_cell = str(int(iw_idx + 1)) + "_" + str(int(obs_idx + 1))
        cell_path = base_path + "results/sim_1/cell_" + sim_cell
        Path(cell_path).mkdir(parents = True, exist_ok = True)
        cell_res_path = cell_path + "/cell_results"
        Path(cell_res_path).mkdir(parents = True, exist_ok = True)
        cell_filename = cell_res_path + "/cell_res"
        try:
            cell_res = load_obj(cell_filename)
        # If the replication results don't exist, conduct replications.
        except FileNotFoundError:
            cell_res = run_replications(n_reps = 100,
                                        n_obs = n_obs,
                                        cov = ref_cov,
                                        loadings = ref_loadings,
                                        intercepts = ref_intercepts,
                                        batch_size = 32,
                                        input_dim = 250,
                                        inference_model_dims = [130],
                                        latent_dim = 5,
                                        n_cats = [5]*50,
                                        learning_rate = 5e-3,
                                        device = device,
                                        log_interval = sim_log_interval,
                                        steps_anneal = 1000,
                                        kwargs = kwargs,
                                        iw_samples = iw_samples)

            # Save cell results.
            save_obj(cell_res, cell_filename)
            print("Saved results for simulation cell " + str(int(iw_idx + 1)) + ", " + str(int(obs_idx + 1)))

Create bias, MSE, fitting time, and scree plots.

In [ ]:
# Load data generating parameters.
ref_loadings   = torch.from_numpy(np.loadtxt(base_path + "data/simulations/loadings.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_intercepts = torch.from_numpy(np.loadtxt(base_path + "data/simulations/intercepts.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_cov        = torch.from_numpy(np.loadtxt(base_path + "data/simulations/lv_cov.txt",
                                  dtype = float,
                                  delimiter = ",")).float()

res_path = base_path + "results/sim_1/"
res_dirnames = ["cell_1_1", "cell_1_2", "cell_1_3", "cell_1_4",
                "cell_2_1", "cell_2_2", "cell_2_3", "cell_2_4",
                "cell_3_1", "cell_3_2", "cell_3_3", "cell_3_4"]

# Extract factor loadings and run times.
for dirname in res_dirnames:
    cell_res = load_obj(res_path + dirname + "/cell_results/cell_res")
    
    # Save loadings.
    Path(res_path + dirname + "/loadings/").mkdir(parents = True, exist_ok = True)
    for idx, loadings in enumerate(cell_res["loadings"]):
        np.savetxt(res_path + dirname + "/loadings/loadings_" + str(idx) + ".txt",
                   loadings,
                   delimiter = ",")
    
    # Save run times.
    Path(res_path + dirname + "/run_times/").mkdir(parents = True, exist_ok = True)
    for idx, run_time in enumerate(cell_res["run_time"]):
        np.savetxt(res_path + dirname + "/run_times/run_time_" + str(idx) + ".txt",
                   np.array(run_time, ndmin = 1),
                   delimiter = ",", fmt = "%f")

# Rotate factor loadings.
subprocess.call(base_path + "code/r/rotate_sim_1.R")

# Get rotated loadings and covariance matrices.
cell_res_ls = []
for dirname in res_dirnames:
    cell_res = load_obj(res_path + dirname + "/cell_results/cell_res")
    cell_res["loadings"] = [np.loadtxt(res_path + dirname + "/rotated_loadings/rotated_loadings_" + str(i) + ".txt",
                                       dtype = float) for i in range(100)]
    cell_res["cov"] = [np.loadtxt(res_path + dirname + "/covs/cov_" + str(i) + ".txt",
                                       dtype = float) for i in range(100)]
    rot_mats = [np.loadtxt(res_path + dirname + "/rot_mats/rot_mat_" + str(i) + ".txt",
                           dtype = float) for i in range(100)]
    cell_res["scores"] = [torch.matmul(torch.from_numpy(rot_mats[i]).T.float(),
                                       torch.from_numpy(cell_res["scores"][i]).T).T.numpy() for
                          i in range(100)]
    cell_res["run_time"] = [np.loadtxt(res_path + dirname + "/run_times/run_time_" + str(i) + ".txt",
                                       dtype = float, skiprows = 1).sum() for i in range(100)]
    cell_res_ls.append(cell_res)

bias = bias_boxplots(cell_res_ls = cell_res_ls,
                     cov = ref_cov,
                     loadings = ref_loadings,
                     intercepts = ref_intercepts,
                     n_cats = [5]*50,
                     ldgs_lims = [-.025, .025],
                     cov_lims = [-.04, .04],
                     int_lims = [-0.13, 0.11],
                     power = 1)
mse = bias_boxplots(cell_res_ls = cell_res_ls,
                    cov = ref_cov,
                    loadings = ref_loadings,
                    intercepts = ref_intercepts,
                    n_cats = [5]*50,
                    power = 2,
                    ldgs_lims = [0, 0.0035],
                    cov_lims = [0, 0.0035],
                    int_lims = [0, 0.055])
bias.show(); mse.show()

time = time_plot(cell_res_ls = cell_res_ls,
                 y_lims = [0, 250])
time.show()

scree = scree_plots(cell_res_ls = cell_res_ls)
scree.show()

# Save plots to a single PDF.
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/bias_plots_sim_1.pdf")
pdf.savefig(bias, dpi = 300)
pdf.close()
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/mse_plots_sim_1.pdf")
pdf.savefig(mse, dpi = 300)
pdf.close()
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/time_plots_sim_1.pdf")
pdf.savefig(time, dpi = 300)
pdf.close()
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/scree_plots_sim_1.pdf")
pdf.savefig(scree, dpi = 300)
pdf.close()

Compute factor score correlations.

In [ ]:
for i in range(len(cell_res_ls)):
    print("Simulation Cell ", int(np.floor((i % 12) / 4) + 1), (i % 4) + 1)
    cors = score_cors(cell_res_ls[i], ref_loadings)
    # One can inspect these correlations more closely -- I print the mean and SD here for brevity.
    print("Mean Correlation: ", np.mean([np.mean(abs(cor)) for cor in cors]))
    print("St. Dev. Correlation: ", np.std([np.mean(abs(cor)) for cor in cors])) 

### Simulation 2: Comparing Estimator Efficiency in the Classical Asymptotic Regime

Save simulated data sets.

In [ ]:
# Load data generating parameters.
ref_loadings   = torch.from_numpy(np.loadtxt(base_path + "data/simulations/loadings.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_intercepts = torch.from_numpy(np.loadtxt(base_path + "data/simulations/intercepts.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_cov        = torch.from_numpy(np.loadtxt(base_path + "data/simulations/lv_cov.txt",
                                  dtype = float,
                                  delimiter = ",")).float()

factor_mul_ls = [2, 2, 2, 2]
item_mul_ls   = [1, 1, 1, 1]
n_obs_ls      = [1000, 2000, 5000, 10000]
n_reps        = 100
orig_n_cats   = 5
new_n_cats    = 5

sim_path = base_path + "data/simulations/sim_2/"
Path(sim_path).mkdir(parents = True, exist_ok = True)

for idx, factor_mul in enumerate(factor_mul_ls):
    Path(sim_path + "cond_" + str(idx)).mkdir(parents = True, exist_ok = True)

    loadings, cov, intercepts = make_gen_params(loadings = ref_loadings,
                                                intercepts = ref_intercepts,
                                                orig_n_cats = orig_n_cats,
                                                new_n_cats = new_n_cats,
                                                cov = ref_cov,
                                                factor_mul = factor_mul,
                                                item_mul = item_mul_ls[idx])

    for rep in range(n_reps):
        # Set random seeds.
        torch.manual_seed(rep + 1)
        np.random.seed(rep + 1)

        # Simulate data set.
        data = sim_mirt(n_obs = n_obs_ls[idx],
                        distribution = dist.MultivariateNormal(torch.zeros(cov.size(0)), cov),
                        loadings = loadings,
                        intercepts = intercepts,
                        n_cats = [new_n_cats]*loadings.shape[0])[0]

        # Save data set.
        if idx == 0 and rep == 78:
            np.savetxt(sim_path + "cond_" + str(idx) + "/rep_" + str(rep) + ".csv",
                       data.numpy(), 
                       delimiter = ",")
        else:
            np.savetxt(sim_path + "cond_" + str(idx) + "/rep_" + str(rep) + ".gz",
                       data.numpy(), 
                       delimiter = ",")

Fit amortized IWVI models.

In [ ]:
res_path = base_path + "results/sim_2/aiwvi/"

# Set simulation cell parameters and regularization hyperparameter grid.
n_obs_ls = [1000, 2000, 5000, 10000]
n_reps   = 100

for obs_idx, n_obs in enumerate(n_obs_ls):
    # Print simulation condtion.
    print("Starting replications for N = " + str(n_obs))

    # Get simulation condtion.
    cond = str(int(obs_idx))

    # Make directory to store results.
    Path(res_path).mkdir(parents = True, exist_ok = True)
    Path(res_path + "cond_" + cond).mkdir(parents = True, exist_ok = True)

    for rep in range(n_reps):
        print("Starting replication", rep)

        # Read in data.
        if obs_idx == 0 and rep == 78:
            data = pd.read_csv(base_path + "data/simulations/sim_2/cond_" + cond + "/rep_" + str(rep) + ".csv",
                   header = None,
                   sep = ",").to_numpy()
        else:
            data = pd.read_csv(base_path + "data/simulations/sim_2/cond_" + cond + "/rep_" + str(rep) + ".gz",
                               header = None,
                               sep = ",").to_numpy()
        sim_loader = torch.utils.data.DataLoader(tensor_dataset(data),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 **kwargs)

        # Some hyperparameters for fitting.
        n_cats = [5]*100
        input_dim = 500
        latent_dim = 10
        inference_model_dims = [int((input_dim + 2 * latent_dim) / 2)]

        # Fit model while ensuring numerical stability.
        nan_output = True
        seed = rep
        while nan_output:
            # Set random seeds.
            torch.manual_seed(seed)
            np.random.seed(seed)

            # Initialize model.
            start = timeit.default_timer()
            vae = MIRTVAEClass(input_dim = input_dim,
                               inference_model_dims = inference_model_dims,
                               latent_dim = latent_dim,
                               n_cats = n_cats,
                               learning_rate = 2.5e-3,
                               device = device,
                               log_interval = sim_log_interval,
                               steps_anneal = 1000)

            # Train model.
            vae.run_training(sim_loader, sim_loader, iw_samples = 5)
            stop = timeit.default_timer()

            # Evaluate model.
            elbo = vae.test(sim_loader,
                            mc_samples = 1,
                            iw_samples = 5,
                            print_result = False)

            if np.isnan(elbo):
                nan_output = True
                seed += 1
            else:
                nan_output = False

        # Save run time.
        Path(res_path + "cond_" + cond + "/run_times").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/run_times/run_time_" + str(rep) + ".txt",
                   np.array([stop - start]), 
                   delimiter = ",",
                   fmt = "%f")

        # Save model loadings.
        Path(res_path + "cond_" + cond + "/loadings").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/loadings/loadings_" + str(rep) + ".txt",
                   vae.model.loadings.weight.data.numpy(),
                   fmt='%f')

        # Save model intercepts.
        Path(res_path + "cond_" + cond + "/ints").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/ints/ints_" + str(rep) + ".txt",
                   vae.model.intercepts.bias.data.numpy(),
                   fmt = "%f")

        # Save model log-likelihood.
        ll = -vae.bic(sim_loader,
                      iw_samples = 100)[1] # I only use 100 IW samples to speed up the simulation.
        Path(res_path + "cond_" + cond + "/lls").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/lls/ll_" + str(rep) + ".txt",
                   np.array([ll]), 
                   delimiter = ",",
                   fmt = "%f")

        print("Stored results for replication", rep)

# Rotate factor loadings.
subprocess.call(base_path + "code/r/rotate_sim_2.R")

Make boxplots of results.

In [ ]:
# Load data generating parameters.
ref_loadings   = torch.from_numpy(np.loadtxt(base_path + "data/simulations/loadings.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_intercepts = torch.from_numpy(np.loadtxt(base_path + "data/simulations/intercepts.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_cov        = torch.from_numpy(np.loadtxt(base_path + "data/simulations/lv_cov.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_loadings, ref_cov, ref_intercepts = make_gen_params(loadings = ref_loadings,
                                                        intercepts = ref_intercepts,
                                                        orig_n_cats = 5,
                                                        new_n_cats = 5,
                                                        cov = ref_cov,
                                                        factor_mul = 2,
                                                        item_mul = 1)

# Make list to store amortized IWVI results.
res_path = base_path + "results/sim_2/aiwvi/"
aiwvi_cell_res_ls = []

# Read in amortized IWVI results.
for cond in range(4):
    keys = ["log_likelihood", "run_time", "loadings", "intercepts", "cov"]
    cell_res = {key : [] for key in keys}
    
    # Read results.
    cell_res["loadings"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/rotated_loadings/rotated_loadings_" + str(num) + ".txt",
                                       dtype = float) for num in range(100)]
    cell_res["intercepts"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/ints/ints_" + str(num) + ".txt",
                                         dtype = float) for num in range(100)]
    cell_res["cov"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/covs/cov_" + str(num) + ".txt",
                                  dtype = float) for num in range(100)]
    cell_res["log_likelihood"] = [-np.loadtxt(res_path + "cond_" + str(cond) + "/lls/ll_" + str(num) + ".txt",
                                             dtype = float).item() for num in range(100)]
    cell_res["run_time"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/run_times/run_time_" + str(num) + ".txt",
                                       dtype = float, skiprows = 1).sum() for num in range(100)]
    
    aiwvi_cell_res_ls.append(cell_res)

# Make list to store MH-RM results.
res_path = base_path + "results/sim_2/mhrm/"
mhrm_cell_res_ls = []

# Read in MH-RM results.
for cond in range(4):
    keys = ["log_likelihood", "run_time", "loadings", "intercepts", "cov"]
    cell_res = {key : [] for key in keys}
    
    # Read results.
    cell_res["loadings"] = [unnormalize_loadings(np.loadtxt((res_path + "cond_" + str(cond) +
                                                             "/loadings/loadings_" + str(num)),
                                                            delimiter = ",", dtype = float)) for
                            num in range(100)]
    cell_res["intercepts"] = [-np.loadtxt(res_path + "cond_" + str(cond) + "/ints/ints_" + str(num),
                                          delimiter = ",", dtype = float) for num in range(100)]
    # Accidentally saved T instead of Phi.
    Ts = [np.loadtxt(res_path + "cond_" + str(cond) + "/covs/cov_" + str(num),
                     delimiter = ",", dtype = float) for num in range(100)]
    cell_res["cov"] = [np.matmul(T.transpose(), T) for T in Ts]
    cell_res["log_likelihood"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/lls/ll_" + str(num),
                                             delimiter = ",", dtype = float).item() for num in range(100)]
    run_times = [np.loadtxt(res_path + "cond_" + str(cond) + "/run_times/run_time_" + str(num),
                                       delimiter = ",", skiprows = 1, dtype = float) for num in range(100)]
    cell_res["run_time"] = [sum([run_time[i] for i in (2, 3, 6)]) for run_time in run_times]
    
    mhrm_cell_res_ls.append(cell_res)

# Fix intercepts.
# NOTE: mirt does not report an intercept if a response category does not appear in the data.
# I manually identified runs where certain response categories were missing and inserted NaNs
# where appropriate.
# for rep in [16, 17, 42, 80, 88, 89, 97]:
#     # Read in data.
#     data = pd.read_csv(path + "data/simulations/sim_2/cond_0/rep_" + str(rep) + ".gz",
#                    header = None,
#                    sep = ",")
#     unique_vals = [data.iloc[:, col].unique() for col in data]
#     if any([len(vals) != 2 for vals in unique_vals]):
#         idxs = [len(vals) != 2 for vals in unique_vals].index(True)
#         ints = mhrm_cell_res_ls[0]["intercepts"][rep].copy()
#         temp_n_cats = [1] + [5]*100
#         for idx in [idxs]:
#             temp_ls = ints.tolist()
#             temp_ls.insert(np.cumsum([n_cat - 1 for n_cat in temp_n_cats])[int(np.floor(idx / 5))], np.nan)
#             ints = np.array(temp_ls)
#         np.savetxt(path + "results/sim_2/mhrm/cond_0/ints/ints_" + str(rep),
#                    ints,
#                    delimiter = ",",
#                    fmt = "%s")
# Read in MH-RM results, again.
# for cond in range(4):
#     keys = ["log_likelihood", "run_time", "loadings", "intercepts", "cov"]
#     cell_res = {key : [] for key in keys}
#     
#     # Read results.
#     cell_res["loadings"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/loadings/loadings_" + str(num),
#                                        delimiter = ",", dtype = float) for num in range(100)]
#     cell_res["intercepts"] = [-np.loadtxt(res_path + "cond_" + str(cond) + "/ints/ints_" + str(num),
#                                           delimiter = ",", dtype = float) for num in range(100)]
#     Ts = [np.loadtxt(res_path + "cond_" + str(cond) + "/covs/cov_" + str(num), # Accidentally saved T instead of Phi.
#                      delimiter = ",", dtype = float) for num in range(100)]
#     cell_res["cov"] = [np.matmul(T.transpose(), T) for T in Ts]
#     cell_res["log_likelihood"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/lls/ll_" + str(num),
#                                              delimiter = ",", dtype = float) for num in range(100)]
#     cell_res["run_time"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/run_times/run_time_" + str(num),
#                                        delimiter = ",", skiprows = 1, dtype = float) for num in range(100)]
    
#     mhrm_cell_res_ls.append(cell_res)
    
mse = mhrm_boxplots(aiwvi_cell_res_ls = aiwvi_cell_res_ls,
                    mhrm_cell_res_ls = mhrm_cell_res_ls,
                    cov = ref_cov,
                    loadings = ref_loadings,
                    intercepts = ref_intercepts,
                    n_cats = [5]*100,
                    ldgs_lims = [0, 0.0016],
                    cov_lims = [0, 0.002],
                    int_lims = [0, 0.025])
mse.show()

nums = [1000, 2000, 5000, 10000]
times = comparison_time_plots(aiwvi_cell_res_ls,
                              mhrm_cell_res_ls,
                              ["N = " + f"{num:,}".replace(',', ' ') for num in nums],
                              "Amortized IWVI", "MH-RM",
                              "Fitting Times for Amortized IWVI vs. MH-RM")
times.show()

# Save plots to a single PDF.
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/mse_plots_sim_2.pdf")
pdf.savefig(mse, dpi = 300)
pdf.close()
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/time_plot_sim_2.pdf")
pdf.savefig(times, dpi = 300)
pdf.close()

### Simulation 3: Comparing Estimator Efficiency in the Double Asymptotic Regime

Save simulated data sets.

In [ ]:
# Load data generating parameters.
ref_loadings   = torch.from_numpy(np.loadtxt(base_path + "data/simulations/loadings.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_intercepts = torch.from_numpy(np.loadtxt(base_path + "data/simulations/intercepts.txt",
                                  dtype = float,
                                  delimiter = ",")).float()
ref_cov        = torch.from_numpy(np.loadtxt(base_path + "data/simulations/lv_cov.txt",
                                  dtype = float,
                                  delimiter = ",")).float()

factor_mul_ls = [2, 2, 2, 2]
item_mul_ls   = [1, 2, 3, 4]
n_obs_ls      = [2000, 10000, 50000, 100000]
n_reps        = 100
orig_n_cats   = 5
new_n_cats    = 2

sim_path = base_path + "data/simulations/sim_3/"
Path(sim_path).mkdir(parents = True, exist_ok = True)

for idx, factor_mul in enumerate(factor_mul_ls):
    Path(sim_path + "cond_" + str(idx)).mkdir(parents = True, exist_ok = True)
    
    torch.manual_seed(1)
    np.random.seed(1)
    loadings, cov, intercepts = make_gen_params(loadings = ref_loadings,
                                                intercepts = ref_intercepts,
                                                orig_n_cats = orig_n_cats,
                                                new_n_cats = new_n_cats,
                                                cov = ref_cov,
                                                factor_mul = factor_mul,
                                                item_mul = item_mul_ls[idx])
    
    # Save generating parameters.
    np.savetxt(sim_path + "cond_" + str(idx) + "_gen_loadings.txt",
               loadings.numpy(),
               fmt = "%f",
               delimiter = ",")
    np.savetxt(sim_path + "cond_" + str(idx) + "_gen_cov.txt",
           cov.numpy(),
           fmt = "%f",
           delimiter = ",")
    np.savetxt(sim_path + "cond_" + str(idx) + "_gen_intercepts.txt",
           intercepts.numpy(),
           fmt = "%f",
           delimiter = ",")
    
    for rep in range(n_reps):
        # Set random seeds.
        torch.manual_seed(rep + 1)
        np.random.seed(rep + 1)

        # Simulate data set.
        data = sim_mirt(n_obs = n_obs_ls[idx],
                        distribution = dist.MultivariateNormal(torch.zeros(cov.size(0)), cov),
                        loadings = loadings,
                        intercepts = intercepts,
                        n_cats = [new_n_cats]*loadings.shape[0])[0]
        
        # Save data set.
        if idx == 0 and rep == 45:
            np.savetxt(sim_path + "cond_" + str(idx) + "/rep_" + str(rep) + ".csv",
               data.numpy(), 
               delimiter = ",")
        else:
            np.savetxt(sim_path + "cond_" + str(idx) + "/rep_" + str(rep) + ".gz",
                       data.numpy(), 
                       delimiter = ",")

Fit amortized IWVI models.

In [ ]:
# Set simulation cell parameters and regularization hyperparameter grid.
n_obs_ls      = [2000, 10000, 50000, 100000]
item_mul_ls   = [1, 2, 3, 4]
lr_ls         = [5e-3, 5e-3, 2.5e-3, 2.5e-3]
n_reps        = 100

for obs_idx, n_obs in enumerate(n_obs_ls):
    # Print simulation condition.
    print("Starting replications for N = " + str(n_obs))

    # Get simulation condition and learning rate.
    cond = str(int(obs_idx))
    lr = lr_ls[obs_idx]

    # Make directory to store results.
    res_path = base_path + "results/sim_3/aiwvi/"
    Path(res_path).mkdir(parents = True, exist_ok = True)
    Path(res_path + "cond_" + cond).mkdir(parents = True, exist_ok = True)

    for rep in range(n_reps):
        print("Starting replication", rep)
        # Read in data.
        if obs_idx == 0 and rep == 45:
            data = pd.read_csv(base_path + "data/simulations/sim_3/cond_" + cond + "/rep_" + str(rep) + ".csv",
                   header = None,
                   sep = ",").to_numpy()
        else:
            data = pd.read_csv(base_path + "data/simulations/sim_3/cond_" + cond + "/rep_" + str(rep) + ".gz",
                               header = None,
                               sep = ",").to_numpy()
        sim_loader = torch.utils.data.DataLoader(tensor_dataset(data),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 **kwargs)

        # Some hyperparameters for fitting.
        item_mul     = item_mul_ls[obs_idx]
        n_cats       = [2] * 100 * item_mul
        input_dim    = 200 * item_mul
        latent_dim   = 10
        inference_model_dims = [int((input_dim + 2 * latent_dim) / 2)]

        # Fit model while ensuring numerical stability.
        nan_output = True
        seed = rep
        while nan_output:
            # Set random seeds.
            torch.manual_seed(seed)
            np.random.seed(seed)

            # Initialize model.
            start = timeit.default_timer()
            vae = MIRTVAEClass(input_dim = input_dim,
                               inference_model_dims = inference_model_dims,
                               latent_dim = latent_dim,
                               n_cats = n_cats,
                               learning_rate = lr,
                               device = device,
                               log_interval = sim_log_interval,
                               steps_anneal = 1000)

            # Fit model.
            vae.run_training(sim_loader, sim_loader, iw_samples = 5)
            stop = timeit.default_timer()

            # Evaluate model.
            elbo = vae.test(sim_loader,
                            mc_samples = 1,
                            iw_samples = 5,
                            print_result = False)

            if np.isnan(elbo):
                nan_output = True
                seed += 1
            else:
                nan_output = False

        # Save run time.
        Path(res_path + "cond_" + cond + "/run_times").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/run_times/run_time_" + str(rep) + ".txt",
                   np.array([stop - start]), 
                   delimiter = ",",
                   fmt = "%f")

        # Save model loadings.
        Path(res_path + "cond_" + cond + "/loadings").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/loadings/loadings_" + str(rep) + ".txt",
                   vae.model.loadings.weight.data.numpy(),
                   fmt='%f')

        # Save model intercepts.
        Path(res_path + "cond_" + cond + "/ints").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/ints/ints_" + str(rep) + ".txt",
                   vae.model.intercepts.bias.data.numpy(),
                   fmt = "%f")

        # Save factor scores.
        Path(res_path + "cond_" + cond + "/scores/").mkdir(parents = True, exist_ok = True)
        np.savetxt(res_path + "cond_" + cond + "/scores/score_" + str(rep) + ".txt",
                   vae.scores(sim_loader, mc_samples = 20, iw_samples = 5).numpy(),
                   fmt = "%f")

        print("Stored results for replication", rep)

# Rotate factor loadings.
subprocess.call(base_path + "code/r/rotate_sim_3.R")

Make boxplots of results.

In [ ]:
# Load data generating parameters.
ref_loadings_ls = []
ref_intercepts_ls = []
for cond in range(4):
    ref_loadings_ls.append(torch.from_numpy(np.loadtxt((base_path + "/data/simulations/sim_3/cond_" +
                                                        str(cond) + "_gen_loadings.txt"),
                                                       dtype = float,
                                                       delimiter = ",")).float())
    ref_intercepts_ls.append(torch.from_numpy(np.loadtxt((base_path + "/data/simulations/sim_3/cond_" +
                                                          str(cond) + "_gen_intercepts.txt"),
                                                         dtype = float,
                                                         delimiter = ",")).float())

# Make list to store amortized IWVI results.
res_path = base_path + "results/sim_3/aiwvi/"
aiwvi_cell_res_ls = []

# Read in amortized IWVI results.
for cond in range(4):
    keys = ["log_likelihood", "run_time", "loadings", "intercepts", "cov"]
    cell_res = {key : [] for key in keys}
    
    # Read results.
    cell_res["loadings"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/rotated_loadings/rotated_loadings_" + str(num) + ".txt",
                                       dtype = float) for num in range(100)]
    cell_res["intercepts"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/ints/ints_" + str(num) + ".txt",
                                         dtype = float) for num in range(100)]
    cell_res["run_time"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/run_times/run_time_" + str(num) + ".txt",
                                       dtype = float, skiprows = 1).sum() for num in range(100)]
    
    aiwvi_cell_res_ls.append(cell_res)

# Make list to store CJMLE results.
res_path = base_path + "results/sim_3/cjmle/"
cjmle_cell_res_ls = []

# Read in CJMLE results.
for cond in range(4):
    keys = ["log_likelihood", "run_time", "loadings", "intercepts", "cov"]
    cell_res = {key : [] for key in keys}
    
    # Read results.
    cell_res["loadings"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/loadings/loadings_" + str(num),
                                       delimiter = ",", dtype = float) for num in range(100)]
    cell_res["intercepts"] = [-np.loadtxt(res_path + "cond_" + str(cond) + "/ints/ints_" + str(num),
                                         delimiter = ",", dtype = float) for num in range(100)]
    cell_res["run_time"] = [np.loadtxt(res_path + "cond_" + str(cond) + "/run_times/run_time_" + str(num),
                                       delimiter = ",", skiprows = 1, dtype = float).sum() for num in range(100)]
    
    cjmle_cell_res_ls.append(cell_res)
    
mse = cjmle_boxplots(aiwvi_cell_res_ls = aiwvi_cell_res_ls,
                     cjmle_cell_res_ls = cjmle_cell_res_ls,
                     loadings_ls = ref_loadings_ls,
                     intercepts_ls = ref_intercepts_ls)
mse.show()

n_obs_ls = [2000, 10000, 50000, 100000]
n_items_ls = [100, 200, 300, 400]
times = comparison_time_plots(aiwvi_cell_res_ls,
                              cjmle_cell_res_ls,
                              ["N = " + f"{n_obs:,}".replace(',', ' ') + ",\nJ = " + f"{n_items:,}".replace(',', ' ') for 
                               n_obs, n_items in zip(n_obs_ls, n_items_ls)],
                              "Amortized IWVI", "CJMLE",
                              "Fitting Times for Amortized IWVI vs. CJMLE")
times.show()

# Save plots to a single PDF.
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/mse_plots_sim_3.pdf")
pdf.savefig(mse, dpi = 300, bbox_inches = "tight")
pdf.close()
pdf = matplotlib.backends.backend_pdf.PdfPages(base_path + "figures/time_plot_sim_3.pdf")
pdf.savefig(times, dpi = 300, bbox_inches = "tight")
pdf.close()